In [1]:
import datetime
from datetime import date
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import tkinter as tk
import tkinter.filedialog as fd
import getpass
import time
import pandas as pd
import tkinter as tk
import easygui
from pathlib import Path
import datetime
import os
import requests
import json
import math
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from PyPDF2 import PdfMerger,PdfReader
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from selenium.webdriver.common.action_chains import ActionChains


In [3]:
def error():
    global df
    now = datetime.datetime.now()
    df.to_excel('errors_'+str(now.strftime("%d%m%Y %H%M%S"))+'.xlsx')


def custom_wait_clickable_and_click(elem, attempts=20):
    count = 0
    a='no success'
    while count < attempts:
        try:
            if(a!='success'):
                elem.click()
                a='success'
            elif(a=='success'):
                break
        except:
            time.sleep(1)
            count = count + 1

def pdf_gen(a,b,c,d,e,f,g,h,i,j):
    my_doc = SimpleDocTemplate("table.pdf", pagesize = letter)
    my_obj = []
    # defining Data to be stored on table
    my_data = [
       ["Information", 'Details'],
       ["EPR Invoice no.", a],
       ["Customer Registeration Type", b],
       ["Entity Type", c],
       ["Name of Entity", d],
       ["Plastic Material Type", e],
       ["Other Plastic Material Type", f],
       ["Category of Plastic", g],
       ["Financial Year", h],
       ["Quantity (TPA)", i],
       ["GST Paid", j],
    ]
    # Creating the table with 3 rows
    my_table = Table(my_data, 1 * [3.5 * inch], 11 * [0.5 * inch])
    # setting up style and alignments of borders and grids
    my_table.setStyle(
       TableStyle(
           [
               
               ("ALIGN", (-1, -1), (-1, -1), "LEFT"),
               ("VALIGN", (-1, -1), (-1, -1), "BOTTOM"),
               ("ALIGN", (-1, -1), (-1, -1), "LEFT"),
               ("VALIGN", (-1, -1), (-1, -1), "BOTTOM"),
               ("INNERGRID", (0, 0), (-1, -1), 1, colors.black),
               ("BOX", (0, 0), (-1, -1), 1, colors.black),
               ('BACKGROUND',(0,0),(1,0),colors.lightblue),
           ]
       )
    )
    my_obj.append(my_table)
    my_doc.build(my_obj)




In [4]:
global errors
global invoicee
global roww
global driver
today = date.today()
try:
    driver = webdriver.Chrome()
except:
    driver = webdriver.Edge()
driver.maximize_window()
driver.implicitly_wait(15)
driver.get('https://eprplastic.cpcb.gov.in/#/plastic/home')
time.sleep(1)

mail = easygui.enterbox("Enter Email ")
passs = easygui.enterbox("Enter Password")
action = ActionChains(driver)
action.click(on_element = driver.find_element(by=By.XPATH, value='//*[@id="username"]')).perform()
action.click(on_element = driver.find_element(by=By.XPATH, value='//*[@id="password"]')).perform()
driver.find_element(by=By.XPATH, value='//*[@id="username"]').send_keys(mail)
driver.find_element(by=By.XPATH, value='//*[@id="password"]').send_keys(passs)


errors = []
invoicee = []
roww=[]
c=-1

In [5]:
global errors
global invoicee
global driver
global roww
driver = driver
login_select = easygui.enterbox('with which id you want to proceed?\na) PRODUCER\nb) BRAND OWNER \nc) IMPORTER')
driver.implicitly_wait(15)
errors = []
invoicee = []
c=-1
start = time.time()
if(login_select.lower() =="a"):
    driver.get('https://eprplastic.cpcb.gov.in/#/epr/producer-list')
    driver.refresh()
    producer()
elif(login_select.lower() =="b"):
    driver.get('https://eprplastic.cpcb.gov.in/#/epr/bo-list')
    driver.refresh()
    brand_owner()
elif(login_select.lower() =="c"):
    driver.get('https://eprplastic.cpcb.gov.in/#/epr/importer-list')
    driver.refresh()
    importer()
else:
    print("PLEASE ENTER CORRECT CHOICE")
end = time.time()
print("The time of execution of program is :",
  (end-start), "s")
error()


NameError: name 'producer' is not defined

In [14]:
# global errors
# global invoicee
# global driver
# global roww
# driver = driver
# login_select = easygui.enterbox('with which id you want to proceed?\na) PRODUCER\nb) BRAND OWNER \nc) IMPORTER')
# driver.implicitly_wait(15)
# errors = []
# invoicee = []
# c=-1
# start = time.time()
# if(login_select.lower() =="a"):
#     driver.get('https://eprplastic.cpcb.gov.in/#/epr/producer-list')
#     driver.refresh()
#     producer()
# elif(login_select.lower() =="b"):
#     driver.get('https://eprplastic.cpcb.gov.in/#/epr/bo-list')
#     driver.refresh()
#     brand_owner()
# elif(login_select.lower() =="c"):
#     driver.get('https://eprplastic.cpcb.gov.in/#/epr/importer-list')
#     driver.refresh()
#     importer()
# else:
#     print("PLEASE ENTER CORRECT CHOICE")
# end = time.time()
# print("The time of execution of program is :",
#   (end-start), "s")
# error()

def producer():
    login_token = driver.get_cookies()[0]["value"]
##    print(login_token)
    headers = {
        'Content-Type': 'application/json',
        'Cookie': f'login-token={login_token}',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    }
    
    errors = []
    
    def entity_type(entity, state, category, material_type):
        url = "https://eprplastic.cpcb.gov.in/epr/api/v1.0/pibo/get_pibo_dropdown_data"
        
        payload = json.dumps({
            "type": [
                "entity_type_for_sales",
                "states_list",
                "plastic_category",
                "plastic_material_type"
            ],
            "company_id": 15247,
            "section": "procurement"
        })

        try:
            response = requests.post(url, headers=headers, data=payload, verify=False)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return None

        try:
            datas = response.json().get("data", {})
            entity_value = next((i["value"] for i in datas.get("entity_type_for_sales", []) if entity.lower() == i["label"].lower()), None)
            state_value = next((i["value"] for i in datas.get("states_list", []) if state.lower() == i["label"].lower()), None)
            category_value = next((i["value"] for i in datas.get("plastic_category", []) if category.lower() == i["label"].lower()), None)
            material_type_value = next((i["value"] for i in datas.get("plastic_material_type", []) if material_type.lower() == i["label"].lower()), None)

            return entity_value, state_value, category_value, material_type_value
        
        except KeyError as e:
            print(f"Error parsing data: {e}")
            return None

    def pdf_path(pdf_path, pdf_filename):

        url = "https://eprplastic.cpcb.gov.in/epr/api/v1.0/pwp/upload_image"
        headers = {
                    'Accept': 'application/json, text/plain, */*',
                    'Cookie': f'login-token={login_token}',
                    }
        payload = {'section': 'upload_filepath'}
        files=[
        ('file',(f"{pdf_filename}.pdf",open(f'{pdf_path}','rb'),'application/pdf'))
        ]
        response = requests.request("POST", url, headers=headers, data=payload, files=files,verify=False)
        path = response.json()["data"]["path"]
        return path

    def entity_register_id(entity_value, name):
        url = "https://eprplastic.cpcb.gov.in/epr/api/v1.0/pibo/list_entity_name"

        payload = json.dumps({
        "entity_type": entity_value
        })

        try:
            response = requests.post(url, headers=headers, data=payload, verify=False)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return None

        try:
            datas = response.json().get("data", [])
            registered_id = next((i["value"] for i in datas if name.lower().strip() == i["label"].lower().strip()), None)
            return registered_id
        
        except KeyError as e:
            print(f"Error parsing data: {e}")
            return None

    select = easygui.enterbox("you want to proceed with?\n a) Plastic Raw Material/Packaging Procured from Non-registered Entity\n b)Plastic Raw material sale to PIBOs")
    root = tk.Tk()
    file = fd.askopenfilename(parent=root, title='Choose a record file')
    root.destroy()

    df = pd.read_excel(file, keep_default_na=False, converters={'pdf_filename': str, 'Bank account no': str})

    df = df.astype(str)
    df.columns = [x.lower() for x in df.columns]
    df['bank account no'] = df['bank account no'].str.strip()
    df = df.fillna(0)
    df = df.replace('', 0).infer_objects(copy=False)
    df['date of invoice'] = df['date of invoice'].astype(str)

    if select.lower() == 'a':
        root = tk.Tk()
        file2 = fd.askopenfilenames(parent=root, title='Choose a pdf files')
        root.destroy()
        df1 = pd.DataFrame(list(file2), columns=['file_path'])
        df1['file_name'] = df1['file_path'].apply(lambda x: x.split("/")[-1].split(".pdf")[0].split(".PDF")[0])

        i = -1
        time.sleep(2)
        while i < len(df) - 1:
            i += 1
            print(i)
            registration_type = df['registration type'][i]
            entity = df['entity type'][i]
            name = df['name of entity'][i]
            state = df['state'][i]
            entity_address = df['address'][i]
            entity_mobile = str(df['mobile number'][i])[:10]
            category = df['category of plastic'][i]
            recycled_plastic = str(df['recycled plastic %'][i])
            financial_year = df['financial year'][i]
            a = str(df['date of invoice'][i])[:8]
            d = a[:4] + '/' + a[4:6] + '/' + a[6:]
            datetime0 = datetime.datetime.strptime(d, '%Y/%m/%d')
            datetime1 = datetime0.date()
            year = datetime.date.strftime(datetime1, '%Y-%m-%d')
            quantity = df['quantity (tpa)'][i]
            gst_no = df['gst number'][i]
            gst_paid = df['gst paid'][i]
            gst_e_invoice = df['invoice number'][i]
            material_type = df['plastic material type'][i]
            other_type = df['other plastic material type'][i]
            pdf_filename = df['pdf_filename'][i]

            try:
                pdf_file_index = df1[df1['file_name'] == df['pdf_filename'][i]].index[0]
            except:
                pdf_file_index = 0

            if pdf_file_index != 0:
                pdf_file = df1['file_path'][pdf_file_index]
                try:
                    invoice = pdf_path(pdf_file, pdf_filename)
                except:
                    invoice = ""
                if invoice:
                    result = entity_type(entity, state, category, material_type)

                    if result:
                        entity_value, state_value, category_value, material_type_value = result
                        url = "https://eprplastic.cpcb.gov.in/epr/api/v1.0/pibo/save_material_procurement_details"

                        payload = json.dumps({
                            "registration_type": registration_type.lower(),
                            "entity_type": entity_value,
                            "registered_entity_id": None,
                            "registration_number": None,
                            "entity_state_id": state_value,
                            "entity_address": entity_address,
                            "entity_mobile": entity_mobile,
                            "plastic_type": material_type_value,
                            "plastic_category": category_value,
                            "financial_year": financial_year,
                            "year": year,
                            "quantity": quantity,
                            "recycled_plastic": int(recycled_plastic) if recycled_plastic else 0,
                            "gst_no": gst_no,
                            "gst_paid": float(gst_paid),
                            "invoice": invoice,
                            "invoice_number": None,
                            "entity_country": "India",
                            "name": name,
                            "address": "",
                            "gst_e_invoice": gst_e_invoice,
                            "user_id": 15677,
                            "company_id": 15677,
                            "other_type": other_type if other_type else "",
                        })

                        response = requests.post(url, headers=headers, data=payload, verify=False)
                        print(response)
                        invoice_number = response.json().get("data", {}).get("invoice_number")
                        print(invoice_number)

                        # Check if invoice number was generated
                        if not invoice_number:
                            errors.append({
                                "gst_e_invoice": gst_e_invoice,
                                "status": "Your data is not uploaded."
                            })
                        else:
                            print(f"Invoice number generated: {invoice_number}")
                else:
                    errors.append({
                                "gst_e_invoice": gst_e_invoice,
                                # "status": "Your data is not uploaded."
                            })
    
    if select.lower() == 'b':
        df['epr invoice number'] = "na"
        i = -1
        time.sleep(2)
        while i < len(df) - 1:
            i += 1
            print(i)
            registration_type = df['registration type'][i]
            entity = df['entity type'][i]
            name = df['name of entity'][i]
            state = df['state'][i]
            entity_address = df['address'][i]
            entity_mobile = str(df['mobile number'][i])[:10]
            category = df['category of plastic'][i]
            recycled_plastic = str(df['recycled plastic %'][i])
            financial_year = df['financial year'][i]
            quantity = str(df['quantity (tpa)'][i])
            gst_no = df['gst number'][i]
            gst_paid = str(df['gst paid'][i])
            gst_e_invoice = df['invoice number'][i]
            material_type = df['plastic material type'][i]
            other_type = df['other plastic material type'][i]
            account_no = df['bank account no'][i]
            ifsc = df['ifsc code'][i]
            cat_1 = df['cat-1 container capacity'][i]
            if cat_1:
                if cat_1.lower() == "Containers > 0.9L and < 4.9 L".lower():
                    container_type = 1
                elif cat_1.lower() == "Containers > 4.9 L".lower():
                    container_type = 2
                elif cat_1.lower() == "Containers < 0.9 L".lower():
                    container_type = 0
                else:
                    container_type = 5
            else:
                container_type = 5

            result = entity_type(entity, state, category, material_type)

            if result:
                entity_value, state_value, category_value, material_type_value = result

            url = "https://eprplastic.cpcb.gov.in/epr/api/v1.0/pibo/save_sale_receipts"
            if registration_type.lower() == "unregistered":
                payload = json.dumps({
                    "type": registration_type.lower(),
                    "entity_type": entity_value,
                    "registered_entity_id": None,
                    "name": name,
                    "address": entity_address,
                    "entity_state_id": state_value,
                    "mobile": int(entity_mobile),
                    "financial_year": financial_year,
                    "gst_number": gst_no,
                    "account_no": account_no,
                    "ifsc": ifsc,
                    "gst_paid": gst_paid,
                    "quantity": quantity,
                    "gst_e_invoice": gst_e_invoice,
                    "plastic_category": category_value,
                    "recycled_plastic": recycled_plastic,
                    "container_type": container_type,
                    "plastic_type": material_type_value,
                    "epr_registration_number": None,
                    "invoice": "",
                    "company_id": 11579,
                    "not_registered": False,
                    "other_type": other_type if other_type else "",
                })
                response = requests.post(url, headers=headers, data=payload, verify=False)
                invoice_number = response.json().get("data", {}).get("invoice_number")

                if not invoice_number:
                    errors.append({
                        "gst_e_invoice": gst_e_invoice,
                        "status": "Your data is not uploaded."
                    })
                else:
                    print(f"Invoice number generated: {invoice_number}")
                
            else:
                registered_entity_id = entity_register_id(entity_value, name)
                if registered_entity_id:
                    payload = json.dumps({
                        "type": registration_type.lower(),
                        "financial_year": financial_year,
                        "account_no": account_no,
                        "ifsc": ifsc,
                        "gst_paid": gst_paid,
                        "quantity": quantity,
                        "gst_e_invoice": gst_e_invoice,
                        "plastic_category": category_value,
                        "recycled_plastic": recycled_plastic,
                        "entity_type": entity_value,
                        "registered_entity_id": registered_entity_id,
                        "container_type": container_type,
                        "plastic_type": material_type_value,
                        "state": state_value,
                        "entity_state_id": state_value,
                        "address": "xyz",
                        "mobile": entity_mobile,
                        "gst_number": "27AAACY3846K1ZX",
                        "epr_registration_number": None,
                        "invoice": "",
                        "company_id": 11579,
                        "not_registered": False,
                        "other_type": other_type if other_type else ""
                    })
                    response = requests.post(url, headers=headers, data=payload, verify=False)
                    invoice_number = response.json().get("data", {}).get("invoice_number")

                    if not invoice_number:
                        errors.append({
                            "gst_e_invoice": gst_e_invoice,
                            "status": "Your data is not uploaded."
                        })
                    else:
                        print(f"Invoice number generated: {invoice_number}")

    # Write errors to a CSV file
    if errors:
        error_df = pd.DataFrame(errors)
        error_file_path = "error_log.csv"
        error_df.to_csv(error_file_path, index=False)
        print(f"Errors logged to {error_file_path}")

global errors
global invoicee
global driver
global roww
driver = driver
login_select = easygui.enterbox('with which id you want to proceed?\na) PRODUCER\nb) BRAND OWNER \nc) IMPORTER')
driver.implicitly_wait(15)
errors = []
invoicee = []
c=-1
start = time.time()
if(login_select.lower() =="a"):
    driver.get('https://eprplastic.cpcb.gov.in/#/epr/producer-list')
    driver.refresh()
    producer()
elif(login_select.lower() =="b"):
    driver.get('https://eprplastic.cpcb.gov.in/#/epr/bo-list')
    driver.refresh()
    brand_owner()
elif(login_select.lower() =="c"):
    driver.get('https://eprplastic.cpcb.gov.in/#/epr/importer-list')
    driver.refresh()
    importer()
else:
    print("PLEASE ENTER CORRECT CHOICE")
end = time.time()
print("The time of execution of program is :",
  (end-start), "s")
error()


C:\Users\KamleshZore\AppData\Local\Temp\ipykernel_17184\6759321.py:125: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', 0).infer_objects(copy=False)


0


C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>
None
1


C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZor

...

<Response [200]>
None
14


C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>
None
15


C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KamleshZore\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eprplastic.cpcb.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>
None
Errors logged to error_log.csv
The time of execution of program is : 21.492061138153076 s
